In [1]:
import cv2
import numpy as np

In [2]:
# file_name = 'opencv_data/video/yolo_01.mp4'
file_name = 'opencv_data/video/obama_01.mp4'
min_confidence = 0.5

In [3]:
# Yolo 모델을 로딩한다
net = cv2.dnn.readNet('opencv_data/yolo/yolov3.weights', 
                     'opencv_data/yolo/yolov3.cfg')

In [4]:
# 인식 가능한 항목들의 리스트를 가져온다
with open('opencv_data/yolo/coco.names' ,'r') as fp:
    classes= [line.strip() for line in fp.readlines()]
classes

['person',
 'bicycle',
 'car',
 'motorbike',
 'aeroplane',
 'bus',
 'train',
 'truck',
 'boat',
 'traffic light',
 'fire hydrant',
 'stop sign',
 'parking meter',
 'bench',
 'bird',
 'cat',
 'dog',
 'horse',
 'sheep',
 'cow',
 'elephant',
 'bear',
 'zebra',
 'giraffe',
 'backpack',
 'umbrella',
 'handbag',
 'tie',
 'suitcase',
 'frisbee',
 'skis',
 'snowboard',
 'sports ball',
 'kite',
 'baseball bat',
 'baseball glove',
 'skateboard',
 'surfboard',
 'tennis racket',
 'bottle',
 'wine glass',
 'cup',
 'fork',
 'knife',
 'spoon',
 'bowl',
 'banana',
 'apple',
 'sandwich',
 'orange',
 'broccoli',
 'carrot',
 'hot dog',
 'pizza',
 'donut',
 'cake',
 'chair',
 'sofa',
 'pottedplant',
 'bed',
 'diningtable',
 'toilet',
 'tvmonitor',
 'laptop',
 'mouse',
 'remote',
 'keyboard',
 'cell phone',
 'microwave',
 'oven',
 'toaster',
 'sink',
 'refrigerator',
 'book',
 'clock',
 'vase',
 'scissors',
 'teddy bear',
 'hair drier',
 'toothbrush']

In [5]:
# 은닉층들의 이름
layer_names = net.getLayerNames()

# 각 신경망의 출력층 이름
output_layers = [layer_names[i[0] - 1] for i in net.getUnconnectedOutLayers()]
# output_layers

In [6]:
# 각 물체의 영역 색상을 랜덤하게 생성한다
colors = np.random.uniform(0, 255, size=(len(classes), 3))

In [7]:
# 동영상을 띄운다
cap = cv2.VideoCapture(file_name)

while True:
    ret, frame = cap.read()
    if frame is None:
        break
        
    # 인식
    # 현재 프레임의 정보를 가져온다
    height, width, channels = frame.shape
    cv2.imshow('video', frame)
    
    # 현재 프레임을 리사이징해서 2진데이터로 만든다.
    # 320x320, 416x416, 609x609
    blob = cv2.dnn.blobFromImage(frame, 0.00392, (416, 416), (0, 0, 0), True, crop=False)

    # 2진 데이터를 넣어서 예측 결과를 가져온다.
    net.setInput(blob)
    outs = net.forward(output_layers)
    
    # 각 신경망이 예측한 결과 묶음의 개수(3개) 만큼 반복한다.
    for out in outs :
        # 예측된 사물의 수 만큼 반복한다.
        for detection in out :
            # 예측 정확도를 가지고 있는 배열을 추출한다.
            scores = detection[5:]
            # 가장 높은 정확도를 가지고 있는 곳의 인덱스를 추출한다.
            class_id = np.argmax(scores)
            #print(class_id)
            #print(collections.Counter(scores))

            # 예측 정확도를 가져온다
            confidence = scores[class_id]
            # print(confidence)

            # 예측 정확도가 설정한 최소 정확도 보다 큰 것만 사용한다
            if confidence > min_confidence:

                # 감지한 물체에 영역을 표시한다
                # 물체의 중앙점 x, y좌표
                center_x = int(detection[0] * width)
                center_y = int(detection[1] * height)

                # 물체의 가로, 세로 길이
                w = int(detection[2] * width)
                h = int(detection[3] * height)

                # 영역의 x, y좌표
                x = int(center_x - w / 2)
                y = int(center_y - h / 2)

                # print(x, y, w, h)

                cv2.rectangle(frame, (x, y), (x+w, y+h), colors[class_id], 2)

                # 물체 이름을 표시한다
                text = f'{classes[class_id]} : {int(confidence * 100)}'
                cv2.putText(frame, text, (x, y-5), cv2.FONT_HERSHEY_PLAIN, 1, colors[class_id], 1)
    
    # 영역이 표시된 프레임을 그린다
    cv2.imshow('yolo', frame)
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cv2.destroyAllWindows()